In [1]:
from ltlcross_wrapper import Modulizer, ResAnalyzer, gather_cumulative, gather_mins
import pandas as pd
pd.set_option("precision",0)
import spot
spot.setup()
from spot.jupyter import display_inline
import datetime

make_tgba   = 'ltl2tgba --deterministic -f %f'

end = '| autfilt --small --tgba > %O' # optimizes and saves result to file
def seminator_cmd(reductions=True, options="",optimalizations=True,version=""):
    if version == "":
        reductions = "--postprocess=1" if reductions else "--postprocess=0"
    else:
        reductions = "" if reductions else "-s0"
    konec = end if optimalizations else "| cat > %O"
    return f"{make_tgba} | seminator{version} {options} {reductions} {konec}"

def empc_cmd(special_transitions=False, no_epsilon=True, accepting=False, optimalizations=True ,version=11):
    # version 8 has --ashu-use-epsilon-transitions disabled by default 
    accepting = '' if not accepting else '--ashu-slim-exploration-special-accepting true'
    special_transitions = '' if not special_transitions else '--ashu-slim-exploration-nonempty-breakpoint true'
    end = "autfilt --small --tgba -F $docas > %O" if optimalizations else "cat $docas > %O"
    #no_epsilon = "--ashu-use-epsilon-transitions"+('' if not no_epsilon else ' true')
    return f"docastom=`mktemp`;docas=`mktemp`;{make_tgba}>$docastom; java -jar ~/automaty/epmc-ashu-v{version}.jar formula2automaton --ashu-input-type hoa --ashu-input-file $docastom {special_transitions} {accepting} --ashu-automaton-type slim --ashu-output-file $docas;{end}"
 
    
sem_pipelines = {
    #"yes.seminator#def" : seminator_cmd(),
    #"no.seminator#def"  : seminator_cmd(reductions=False),

    "yes.seminator#slim": seminator_cmd(options="--slim --pure"),
    "yes.seminator#weakslim" : seminator_cmd( options="--slim --weak --pure"),
    "yes.seminator#bestslim" : seminator_cmd( options="--slim --best --pure", optimalizations=False), # seminator already optimizes
    "yes.seminator#bestslim#viatba" : seminator_cmd( options="--slim --best --pure --via-tba", optimalizations=False),
    #"yes.empc#specialslim":empc_cmd(special_transitions=True),
    #"yes.empc#slim":empc_cmd(),
    #"yes.empc#specialslim#acc":empc_cmd(special_transitions=True, accepting=True),
    #"yes.empc#slim#acc":empc_cmd(accepting=True),
    #"no.seminator#slim": seminator_cmd(optimalizations=False, options="--slim --pure"),
    #"no.seminator#weakslim" : seminator_cmd(optimalizations=False, options="--slim --weak --pure"),
    #"no.empc#specialslim":empc_cmd(optimalizations=False, special_transitions=True),
    #"no.empc#slim":empc_cmd(optimalizations=False),
    #"no.empc#specialslim#acc":empc_cmd(optimalizations=False, special_transitions=True, accepting=True),
    #"no.empc#slim#acc":empc_cmd(optimalizations=False, accepting=True),
}


In [2]:
tools = sem_pipelines.copy()
for name, cmd in tools.items():
    print(f"{name}: {cmd}")

yes.seminator#slim: ltl2tgba --deterministic -f %f | seminator --slim --pure --postprocess=1 | autfilt --small --tgba > %O
yes.seminator#weakslim: ltl2tgba --deterministic -f %f | seminator --slim --weak --pure --postprocess=1 | autfilt --small --tgba > %O
yes.seminator#bestslim: ltl2tgba --deterministic -f %f | seminator --slim --best --pure --postprocess=1 | cat > %O
yes.seminator#bestslim#viatba: ltl2tgba --deterministic -f %f | seminator --slim --best --pure --via-tba --postprocess=1 | cat > %O


In [4]:
print("started ", datetime.datetime.now().strftime("%H:%M:%S"))
!mkdir -p data
for s in ["literature", "random"]:
    m = Modulizer(tools, f"../formulae/{s}_nd.ltl", 
                  name=f"{s}_nd", tmp_dir=f"data/{s}_nd.parts",
                  processes=6, timeout="30")
    m.run()
    !mv {s}_nd* data
print("finished ", datetime.datetime.now().strftime("%H:%M:%S"))

started  01:58:50
finished  02:02:56


hi :)

In [5]:
nd_benchmarks = {}
for name in ["literature_nd","random_nd"]:
    b = ResAnalyzer(f"data/{name}.csv", cols=["states","time","acc","transitions"])
    nd_benchmarks[name] = b

In [6]:
tool_set = ["no.owl#best","yes.owl#best","yes.seminator-1-1","no.seminator#def"]
slim = ["yes.seminator#slim","yes.seminator#weakslim"]
slimbest = ["yes.seminator#bestslim","yes.seminator#bestslim#viatba"]
epmc = ["yes.empc#slim", "yes.empc#specialslim","yes.empc#slim#acc", "yes.empc#specialslim#acc"]
tool_set = slim + epmc 
tool_set_raw = []
for tool in tool_set:
    tool_set_raw.append(tool.replace("yes","no"))
tool_set+=slimbest
print(tool_set)365 	

['yes.seminator#slim', 'yes.seminator#weakslim', 'yes.empc#slim', 'yes.empc#specialslim', 'yes.empc#slim#acc', 'yes.empc#specialslim#acc', 'yes.seminator#bestslim', 'yes.seminator#bestslim#viatba']


In [7]:
gather_cumulative(nd_benchmarks, tool_set=tool_set)

,literature_nd,random_nd
tool,,
yes.seminator#bestslim,402,8434
yes.seminator#bestslim#viatba,365,7456
yes.seminator#slim,408,8793
yes.seminator#weakslim,588,10317


In [8]:
gather_cumulative(nd_benchmarks, tool_set=slim)

,literature_nd,random_nd
tool,,
yes.seminator#slim,408,8793
yes.seminator#weakslim,588,10317


### Minimal automata

The following table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools selected by `tool_set` from cell `[5]`. The number in the column **min hits** shows how many times the same size as the smallest automaton was achieved. The number in **unique min hits** counts only cases where the given tool is the only tool with such a small automaton.

In [9]:
gather_mins(nd_benchmarks, tool_set=tool_set)

/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


In [13]:
gather_mins(nd_benchmarks, tool_set=slimbest)

/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  return pd.np.nan
/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  return pd.np.nan
/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  return pd.np.nan
/usr/local/lib/python3.8/site-packages/ltlcross_wrapper-0.7.3-py3.8.egg/ltlcross_wrapper/analyzer.py:358: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy dire

### No optimalizations

In [10]:
gather_cumulative(nd_benchmarks, tool_set=tool_set_raw)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

### Scatter plots

In [12]:
b.bokeh_scatter_plot("yes.seminator#bestslim", "yes.seminator#bestslim#viatba", include_equal=False)
b.bokeh_scatter_plot("yes.seminator#slim", "yes.seminator#bestslim#viatba", include_equal=False)
b.bokeh_scatter_plot("yes.seminator#weakslim", "yes.seminator#bestslim#viatba", include_equal=False)
b.bokeh_scatter_plot("yes.seminator#weakslim", "yes.empc#slim", include_equal=False)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

KeyError: 'yes.empc#slim'